   ## Fuel consumption ratings - 2025 Fuel Consumption Ratings (2025-05-20)

## Project Goal

Analyze the 2025 Canadian Fuel Consumption Ratings dataset and build a regression model to predict CO₂ emissions based on car specifications such as engine size, cylinders, fuel type, and fuel consumption.

### Problem Statement

Cars contribute significantly to greenhouse gas emissions. Given a car’s specifications, can we predict its CO₂ emissions?

**Target variable:** Emissions de CO2 (g/km)

**Predictor variables:** Engine size, cylinders, fuel consumption, fuel type, transmission, etc.

### Loading the libraries:

 We will need to have the following packages:

-  NumPy
-  Matplotlib
-  Pandas
-  Scikit-learn

In [ ]:
!pip install shap


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

In [ ]:
import urllib.request
import json
import pandas as pd

# API endpoint
url = "https://open.canada.ca/data/en/api/3/action/datastore_search?resource_id=2e1a460f-464d-44b7-b711-8870a6eef7b9&limit=5000"

try:
    response = urllib.request.urlopen(url)

    # Check HTTP status
    if response.status == 200:
        raw_data = response.read().decode('utf-8')   # Read ONLY once
        data = json.loads(raw_data)                  # Parse JSON

        # Extract records
        records = data['result']['records']

        # Convert to DataFrame
        df_raw = pd.DataFrame(records)

        print("Data loaded successfully")
        print(df_raw.head())

    else:
        print("HTTP Error:", response.status)

except Exception as e:
    print("Error occurred:", e)


### Recommended Enhancements
1. Add pagination (the API returns 100 rows by default)
The dataset may exceed 5000 rows. The CKAN API uses offset for pagination.

I can give you a full pagination loop if you want.

In [ ]:
list(df_raw.columns)

In [ ]:
df_raw.head()

In [ ]:
### creating a working copy
df_work = df_raw.copy()


### Inspect the data

In [ ]:
df_work.info()

In [ ]:
df_work.isnull().sum()

In [ ]:
### Check unique values for categorical columns:
for col in df_work.select_dtypes(include='object').columns:
    print(col, df_work[col].unique()[:5])  # first 5 unique values


### Drop Unnecessary Columns

#### Identify columns not useful for regression:

In [ ]:
df_work['Annee modele'].unique()

Since it the Annee modele has only one year 2025 , This column won’t help in regression, we can drop it.

Other columns like 'Marque' ,''Categorie de vehicule','Transmission','Type de carburant' have to be one-hot encoded for the catogorical to numerical.

When you might include Marque

If you want to capture brand-specific effects (e.g., some brands are generally more fuel-efficient)

If you have a large dataset where encoding brands won’t create too many columns

If your goal is brand-level insights

## What to drop?
-  '_id'                ✅ (Correct — useless ID column)
-  'Annee modele'       ⚠ Depends (Model year — may matter)
-  'Marque'             ⚠ Brand may influence emissions
-  'Modele'             ⚠ High-cardinality — usually drop
-  'Transmission'       ⚠ Can affect fuel efficiency
-  'Type de carburant'  ❌ IMPORTANT feature — don’t drop
-  'Indice de CO2'      ✅ Correct to drop (derived index)
-  'Indice de smog'     ✅ Safe to drop


In [ ]:
cols_to_drop = ['Annee modele','Marque', 'Route (L/100 km)', 'Combinee (mi/gal)',
                '_id', 'Modele', 'Indice de CO2', 'Indice de smog']

# Drop only if they exist
cols_to_drop = [col for col in cols_to_drop if col in df_work.columns]
df_work = df_work.drop(cols_to_drop, axis=1)


In [ ]:
list(df_work.columns)

### Convert Numeric Columns

In [ ]:
numeric_cols = ['Cylindree (L)', 'Cylindres', 'Ville (L/100 km)',
                'Combinee (L/100 km)', 'Emissions de CO2 (g/km)']

for col in numeric_cols:
    df_work[col] = df_work[col].astype(str).str.replace(',', '.')
    df_work[col] = pd.to_numeric(df_work[col], errors='coerce')


In [ ]:
for col in numeric_cols:
    print(col, df_work[col].unique())

In [ ]:
df_work[numeric_cols].describe()


### Encode Categorical Variables

In [ ]:
### Columns to encode using one-hot encoding:
df_encoded = pd.get_dummies(df_work, 
                             columns=['Categorie de vehicule', 'Transmission', 'Type de carburant'], 
                             drop_first=True)  # drop_first avoids multicollinearity


In [ ]:
print("Shape after encoding:", df_encoded.shape)


In [ ]:
df_work.shape

**Original vs Encoded**

Original df_work shape: (650, 9)

5 numeric columns + 3 categorical columns (approx.)

After one-hot encoding: (650, 40)

Each categorical column got split into multiple 0/1 columns

So 9 → 40 total features (including numeric columns)

### Compute the Correlation Matrix

In [ ]:
df_encoded[numeric_cols].corr()


### Intro and Report
This report summarizes the correlation structure among the key numeric variables in the 2025 Fuel Consumption Ratings dataset. Understanding these relationships is essential for feature selection, model design, and interpreting the mechanical drivers of fuel consumption and CO₂ emissions.

The variables analyzed include:

Cylindree (L) — engine displacement

Cylindres — number of cylinders

Ville (L/100 km) — city fuel consumption

Combinee (L/100 km) — combined fuel consumption

Emissions de CO₂ (g/km) — CO₂ emissions

A correlation matrix was computed using Pearson’s correlation coefficient.

### INTERPRETATION

The correlation structure reveals a coherent mechanical story:

- 1.Fuel consumption and CO₂ emissions are tightly linked.

- 2.City consumption is the strongest driver of combined consumption.

- 3.Engine displacement and cylinder count influence fuel consumption but with variability.

The dataset is clean, consistent, and free of anomalies.

This correlation profile is ideal for predictive modelling, especially for:

- Random Forest

- Gradient Boosting

- XGBoost

- Linear Regression with regularization

### Feature Reduction After Correlation Analysis
Your correlation matrix showed:

Engine size ↔ Cylinders = 0.93  
→ These two features carry almost the same information.

City ↔ Combined = 0.989  
→ These two are nearly identical.

Combined ↔ CO₂ = 0.989  
→ These two are also nearly identical.

This means you have redundant features — multiple columns describing the same underlying physical property.

So we might remove some features before modelling.
After elimination, you’ll have fewer features with less redundancy.
The model won’t suffer from multicollinearity.
You retain interpretability:
cylindree_(l) for engine size,
combinee_(l/100_km) (or emissions) for efficiency

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Compute correlation for numeric features only
corr_matrix = df_encoded.corr(numeric_only=True)

# Plot heatmap
plt.figure(figsize=(12,8))
sns.heatmap(corr_matrix, annot=False, cmap='coolwarm')
plt.title("Correlation Heatmap")
plt.show()


| Feature               | Correlation    | Interpretation                                                                                          |
| --------------------- | -------------- | ------------------------------------------------------------------------------------------------------- |
| `Combinee (L/100 km)` | ~0.99          | The combined fuel consumption is almost directly proportional to CO₂ emissions — makes sense physically |
| `Cylindree (L)`       | ~0.78          | Engine size: bigger engines → more CO₂                                                                  |
| `Cylindres`           | ~0.77          | More cylinders → higher emissions                                                                       |
| `Ville (L/100 km)`    | ~0.74 (approx) | City fuel consumption → higher city usage → more CO₂                                                    |


In [ ]:
df_encoded.columns

In [ ]:
df_model = df_encoded.drop(
    columns=[
        "Cylindres",
        "Ville (L/100 km)",
        "Marque"
        # "Emissions de CO2 (g/km)"  # optional
    ],
    errors="ignore"
)

df_model.shape, df_model.columns


In [ ]:
df_model.head(5)

### Scatterplot: Engine Size vs Combined Fuel Consumption

In [ ]:
plt.figure(figsize=(8,6))
sns.scatterplot(
    data=df_model,
    x="Cylindree (L)",
    y="Combinee (L/100 km)",
    alpha=0.7
)
plt.title("Engine Size vs Combined Fuel Consumption")
plt.xlabel("Engine Displacement (L)")
plt.ylabel("Combined Fuel Consumption (L/100 km)")
plt.show()


In [ ]:
### Scatterplot: CO₂ Emissions vs Combined Fuel Consumption
plt.figure(figsize=(8,6))
sns.scatterplot(
    data=df_model,
    x="Emissions de CO2 (g/km)",
    y="Combinee (L/100 km)",
    alpha=0.7,
    color="tomato"
)
plt.title("CO₂ Emissions vs Combined Fuel Consumption")
plt.xlabel("CO₂ Emissions (g/km)")
plt.ylabel("Combined Fuel Consumption (L/100 km)")
plt.show()


### 1. Define Target + Features

In [ ]:
df_model = df_model.copy()  # ensure clean copy

target = "Combinee (L/100 km)"   # or "Emissions de CO2 (g/km)" if you prefer
X = df_model.drop(columns=[target])
y = df_model[target]


### Do we need scaling?

Linear Regression is sensitive to feature scale, so StandardScaler improves stability and coefficient interpretation.

Tree-based models (Random Forest, Gradient Boosting, XGBoost) do not require scaling because they split on thresholds, not distances.

Therefore:
- Use **scaled features** for Linear Regression.
- Use **unscaled features** for tree-based models.


### 2. Train/Test Split

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


### Baseline Model: Linear Regression

In [ ]:
from sklearn.preprocessing import StandardScaler

# IMPORTANT: Scale AFTER the train/test split, fit ONLY on training data
# (prevents data leakage from test set influencing the scaler)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)   # fit + transform training data
X_test_scaled  = scaler.transform(X_test)         # transform test data ONLY


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

lr = LinearRegression()
lr.fit(X_train_scaled, y_train)          # use scaled features

y_pred_lr = lr.predict(X_test_scaled)    # predict on scaled test set

print("Linear Regression R²:  ", round(r2_score(y_test, y_pred_lr), 4))
print("Linear Regression RMSE:", round(np.sqrt(mean_squared_error(y_test, y_pred_lr)), 4))


In [ ]:
# Metrics already printed in the cell above.
# Kept here as a reference for manual RMSE calculation:
#   mse  = mean_squared_error(y_test, y_pred_lr)
#   rmse = np.sqrt(mse)


### Note:
Combined fuel consumption (L/100 km)

Typical values range from 5 to 20 L/100 km.

An RMSE of 0.2376 means your model’s average prediction error is:

less than 0.25 L/100 km

about 1–2% error

essentially engineering‑grade accuracy

This is exceptionally low for real‑world fuel consumption data.

> **Note:** The snippet below (`df.iloc[:,[0,1]]`) references a variable `df` that 
> doesn't exist in this notebook. It appears to be leftover scratch code from an earlier 
> experiment. It has been kept as a markdown note and should not be executed.
>
> ```python
> X = df.iloc[:,[0,1]].to_numpy()
> y = df.iloc[:,[2]].to_numpy()
> ```

In [ ]:
# NOTE: This cell was part of an older pipeline draft.
# The active pipeline (cells above) already handles scaling correctly.
# Kept for reference only — do not re-run independently.

# from sklearn import preprocessing
# std_scaler = preprocessing.StandardScaler()
# X_std = std_scaler.fit_transform(X)


In [ ]:
# pd.DataFrame(X_std).describe().round(2)
# Commented out — X_std is no longer defined in the active pipeline.


In [ ]:
# This duplicate split/scale block has been superseded by the active pipeline.
# The correct approach (split first → scale on train only → transform test) is
# already implemented in the cells above.


In [ ]:
# This legacy regression block used unscaled X_train — superseded.
# The active LinearRegression model is trained in the cell above using X_train_scaled.


### Stronger Model: Random Forest Regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Random Forest handles non-linear relationships and feature interactions.
# Does NOT require feature scaling.
rf = RandomForestRegressor(n_estimators=300, random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)          # use unscaled X_train (RF is scale-invariant)

y_pred_rf = rf.predict(X_test)


In [ ]:
from sklearn.metrics import mean_squared_error
import numpy as np

mse_rf  = mean_squared_error(y_test, y_pred_rf)
rmse_rf = np.sqrt(mse_rf)

print("Random Forest R²:  ", round(r2_score(y_test, y_pred_rf), 4))
print("Random Forest RMSE:", round(rmse_rf, 4))


In [ ]:
from sklearn.metrics import mean_squared_error
import numpy as np

mse_rf = mean_squared_error(y_test, y_pred_rf)
rmse_rf = np.sqrt(mse_rf)

print("Random Forest R²:", r2_score(y_test, y_pred_rf))
print("Random Forest RMSE:", rmse_rf)


### Note:
Why the random forest is lower than the linear regression?
Highly linear,Mechanically deterministic,Low‑noise,Perfectly encoded,Redundant features removed

### Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

gbr = GradientBoostingRegressor(random_state=42)
gbr.fit(X_train, y_train)         # Gradient Boosting is also scale-invariant

y_pred_gbr = gbr.predict(X_test)

print("Gradient Boosting R²:  ", round(r2_score(y_test, y_pred_gbr), 4))


In [ ]:
from sklearn.metrics import mean_squared_error
import numpy as np

mse_gbr  = mean_squared_error(y_test, y_pred_gbr)
rmse_gbr = np.sqrt(mse_gbr)

print("Gradient Boosting RMSE:", round(rmse_gbr, 4))


### Feature Importance (Random Forest)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

importances = pd.Series(rf.feature_importances_, index=X.columns)
importances.sort_values(ascending=False).head(15).plot(kind='bar', figsize=(10,5))
plt.title("Top 15 Feature Importances (Random Forest)")
plt.show()


Random Forest sees this and says:

“I don’t need anything else. This one feature gives me almost the entire signal.”

That’s why its importance is close to 1.0 and everything else is near zero.

In [ ]:
### SHAP Explainability

import shap

# Use TreeExplainer for Random Forest (fast, exact)
explainer    = shap.TreeExplainer(rf)
shap_values  = explainer.shap_values(X_train)

# Summary plot: global feature importance + direction of effect
shap.summary_plot(shap_values, X_train, feature_names=X_train.columns.tolist())


In [ ]:
import shap

explainer = shap.TreeExplainer(rf)
shap_values = explainer.shap_values(X_train)

shap.summary_plot(shap_values, X_train)


In [ ]:
import pandas as pd

# ── Model Comparison Summary ─────────────────────────────────────────────────
results = pd.DataFrame({
    'Model':  ['Linear Regression', 'Random Forest', 'Gradient Boosting'],
    'R²':     [round(r2_score(y_test, y_pred_lr), 4),
               round(r2_score(y_test, y_pred_rf), 4),
               round(r2_score(y_test, y_pred_gbr), 4)],
    'RMSE':   [round(np.sqrt(mean_squared_error(y_test, y_pred_lr)), 4),
               round(np.sqrt(mean_squared_error(y_test, y_pred_rf)), 4),
               round(np.sqrt(mean_squared_error(y_test, y_pred_gbr)), 4)],
})

results = results.sort_values('RMSE').reset_index(drop=True)
results.index += 1
print(results.to_string())
